In [3]:
# !pip install kfp -U -q --user 

In [4]:
# !pip install google-cloud-aiplatform -U -q --user 

In [5]:
# !pip install google-auth

In [6]:
# !pip install --upgrade google-api-python-client
# !pip install google-cloud

In [ ]:
# bigquery-public-data.ml_datasets.ulb_fraud_detection

In [1]:
from google.cloud import aiplatform

from google.cloud import bigquery as bq
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

In [8]:
from google.oauth2 import service_account


PROJECT_ID = "vertex-ai-tuto-380714"
REGION = "us-central1"
VAR_TARGET = "Class"
credentials = service_account.Credentials.from_service_account_file("vertex-ai-tuto-380714-811aa46f0cfa.json")


In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)
bigquery = bq.Client(credentials=credentials)

In [6]:
# !pip install db-dtypes

In [4]:
pred = bigquery.query(query = f"SELECT * FROM bigquery-public-data.ml_datasets.ulb_fraud_detection LIMIT 10").to_dataframe()

In [6]:
pred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282.0,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,14332.0,1.071950,0.340678,1.784068,2.846396,-0.751538,0.403028,-0.734920,0.205807,1.092726,...,-0.169632,-0.113604,0.067643,0.468669,0.223541,-0.112355,0.014015,0.021504,0.0,0
2,32799.0,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,0.125514,0.480049,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0
3,35799.0,-0.769798,0.622325,0.242491,-0.586652,0.527819,-0.104512,0.209909,0.669861,-0.304509,...,0.152738,0.255654,-0.130237,-0.660934,-0.493374,0.331855,-0.011101,0.049089,0.0,0
4,36419.0,1.047960,0.145048,1.624573,2.932652,-0.726574,0.690451,-0.627288,0.278709,0.318434,...,0.078499,0.658942,-0.067810,0.476882,0.526830,0.219902,0.070627,0.028488,0.0,0


In [11]:
newob = pred[pred.columns[~pred.columns.isin([VAR_TARGET])]].to_dict(orient='records')[0]
# newob

In [12]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

# Get Predictions: Python Client

In [16]:
# firstml is the name of DEPLOYMENT
endpoint = aiplatform.Endpoint.list(filter=f'display_name=firstml')[0]
endpoint.display_name

'firstml'

In [24]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
print(f"The predicted class is {prediction.predictions[0]['classes'][np.argmax(prediction.predictions[0]['scores'])]}")

The predicted class is 0
